# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [65]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [66]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [67]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [68]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

In [69]:
# view in a dataframe
df = pd.read_csv("event_datafile_new.csv")
df.head(3)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88


In [70]:
# view data types for table creation below
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


#### Creating a Cluster

In [71]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [72]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify
                    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [73]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


**Create table for query 1:**

In [74]:
# check for existing table
dropme = "DROP TABLE IF EXISTS music_library"

try:
    session.execute(dropme)
except Exception as e:
    print(e)

## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_library"

query = query + "(artist text, song text, length double, sessionId int, itemInSession int, PRIMARY KEY(sessionId, itemInSession))" 
#query = query + "(sessionId TEXT, itemInSession TEXT, userId TEXT, artist TEXT, song TEXT, length TEXT, PRIMARY KEY((sessionId), itemInSession) )"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
         

**Insert data for query 1:**

In [75]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES(%s, %s,%s, %s,%s)"
        
## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#session.execute(query, (line[8], line[3], line[0], line[9], line[5]))
session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Question 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [76]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

**Create the table for Question 2:**

In [78]:
# check for existing table
dropme = "DROP TABLE IF EXISTS music_library_q2"

try:
    session.execute(dropme)
except Exception as e:
    print(e)


## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS music_library_q2"

query2 = query2 + "(artist text, song text, itemInSession int, userId int, sessionId int, firstName text, \
                    lastName text, PRIMARY KEY((userId, sessionId), itemInSession))" 

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)                

**Insert the data for Question 2:**

In [79]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_q2(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES(%s, %s,%s, %s,%s, %s, %s)"
        
## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [80]:
## Query 2: sessionid, itemInSession, artist, song, firstname, lastname
query_2 = "SELECT artist, song, firstName, lastName FROM music_library_q2 WHERE userid = 10 AND sessionid = 182 "

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstName, row.lastName)

**Create table for query 3:**

In [81]:
# check for existing table
dropme = "DROP TABLE IF EXISTS music_library_q3"

try:
    session.execute(dropme)
except Exception as e:
    print(e)

## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS music_library_q3"

query3 = query3 + "(song text, userId int, sessionId int, firstName text, \
                    lastName text, PRIMARY KEY(song, userId, sessionId))" 

try:
    rows = session.execute(query3)
except Exception as e:
    print(e) 

                    

In [82]:
# Insert the data for q3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_q3(song, userId, sessionId, firstName, lastName)"
        query = query + "VALUES(%s, %s,%s, %s,%s)"
        
## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
session.execute(query, (line[9], int(line[10]), int(line[8]), line[1], line[4]))

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [83]:
# question 3
query_3 = "SELECT firstName, lastName FROM music_library_q3 WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstName, row.lastName)

### Drop the tables before closing out the sessions

In [63]:
## TO-DO: Drop the table before closing out the session
drop_query1 = "DROP TABLE music_library"

try:
    session.execute(drop_query1)
except Exception as e:
    print(e)

In [64]:
## TO-DO: Drop the table before closing out the session
drop_query2 = "DROP TABLE music_library_q2"

try:
    session.execute(drop_query2)
except Exception as e:
    print(e)

In [42]:
## TO-DO: Drop the table before closing out the session
drop_query3 = "DROP TABLE music_library_q3"

try:
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [43]:
session.shutdown()
cluster.shutdown()